In [1]:
%%bash
mkdir -p /kaggle/working/repos
cd /kaggle/working/repos


In [2]:
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import classification_report
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt
from torchvision import models

In [3]:
%%bash
cd /kaggle/working/repos

# Danh sách repo GitHub
github_list=(
  https://github.com/GwLewis369/Hand-written-digit-classification-data.git
https://github.com/gbaonr/CS114_handwritten_digits_data
https://github.com/adamwhite625/CS114_hand_written_digit.git
https://github.com/theRaven1312/CS114.P21
https://github.com/Khoiisme1905/CS114.git
https://github.com/ProjectHT1/machinelearning
https://github.com/Searching96/hand-written-digit.git
https://github.com/Salmon1605/CS114
https://github.com/anhtuann1224/hand_written_digit
https://github.com/NATuanAN/Hand_written_digit_classification_data.git
https://github.com/huapogba/may-hoc
https://github.com/hieutran890j2/CS114.git
https://github.com/votanhoang483/CS114.P21-Hand_written_digit_classification
https://github.com/DHPh/CS114_hand_written_digit/
https://github.com/thaituanUIT/ReminiScenceAI
https://github.com/anngyn/CS114-Hand-Written-Digit
https://github.com/lngphgthao/cs114-hand-written-digit-classification.git
https://github.com/Toan02Ky-UIT/CS114
https://github.com/Lochke/CS114_Handwritten_Digit_Classification.git
https://github.com/toanlamdata/digit-recognition-group
https://github.com/Nohenshin/CS114.P21-2025-
https://github.com/hmcslearning/ML1142025
https://github.com/lngphgthao/cs114-hand-written-digit-classification
https://github.com/huapogba/may-hoc
https://github.com/23520276/Hand-written-digit-classification/
https://github.com/NThong325/CS114
)

for url in "${github_list[@]}"; do
  # Trích xuất tên thư mục từ URL (ví dụ: user_repo)
  user_repo=$(echo "$url" | sed -E 's|https://github.com/([^/]+)/([^/]+)(\.git)?|\1_\2|')
  target="./$user_repo"

  echo "--------------------------------------------------"
  echo "Đang xử lý repo: $url"

  if [[ -d "$target" ]]; then
    echo "Thư mục '$target' đã tồn tại, bỏ qua."
    continue
  fi

  if git clone "$url" "$target"; then
    echo "✅ Clone thành công vào thư mục '$user_repo'"
  else
    echo "❌ Clone thất bại repo '$user_repo'. Vui lòng xem lỗi chi tiết ở trên."
  fi
done

echo "--------------------------------------------------"
echo "Hoàn tất."

--------------------------------------------------
Đang xử lý repo: https://github.com/GwLewis369/Hand-written-digit-classification-data.git
✅ Clone thành công vào thư mục 'GwLewis369_Hand-written-digit-classification-data.git'
--------------------------------------------------
Đang xử lý repo: https://github.com/gbaonr/CS114_handwritten_digits_data
✅ Clone thành công vào thư mục 'gbaonr_CS114_handwritten_digits_data'
--------------------------------------------------
Đang xử lý repo: https://github.com/adamwhite625/CS114_hand_written_digit.git
✅ Clone thành công vào thư mục 'adamwhite625_CS114_hand_written_digit.git'
--------------------------------------------------
Đang xử lý repo: https://github.com/theRaven1312/CS114.P21
✅ Clone thành công vào thư mục 'theRaven1312_CS114.P21'
--------------------------------------------------
Đang xử lý repo: https://github.com/Khoiisme1905/CS114.git
✅ Clone thành công vào thư mục 'Khoiisme1905_CS114.git'
------------------------------------------

Cloning into './GwLewis369_Hand-written-digit-classification-data.git'...
Cloning into './gbaonr_CS114_handwritten_digits_data'...
Cloning into './adamwhite625_CS114_hand_written_digit.git'...
Cloning into './theRaven1312_CS114.P21'...
Cloning into './Khoiisme1905_CS114.git'...
Cloning into './ProjectHT1_machinelearning'...
Updating files: 100% (314/314), done.
Cloning into './Searching96_hand-written-digit.git'...
Cloning into './Salmon1605_CS114'...
Cloning into './anhtuann1224_hand_written_digit'...
Updating files: 100% (407/407), done.
Cloning into './NATuanAN_Hand_written_digit_classification_data.git'...
Cloning into './huapogba_may-hoc'...
Cloning into './hieutran890j2_CS114.git'...
Cloning into './votanhoang483_CS114.P21-Hand_written_digit_classification'...
Updating files: 100% (624/624), done.
Cloning into './DHPh_CS114_hand_written_digit'...
Cloning into './thaituanUIT_ReminiScenceAI'...
Cloning into './anngyn_CS114-Hand-Written-Digit'...
Cloning into './lngphgthao_cs114-han

In [4]:
data_dir = '/kaggle/working/repos'

In [ ]:
!pip install pyheif pillow-heif -q

In [ ]:
import os
import shutil
import random
from collections import defaultdict
from PIL import Image, UnidentifiedImageError
import pyheif
import zipfile
import re
#================================================================
# --- A. CÁC HÀM TIỆN ÍCH ---
#================================================================
# (Các hàm is_valid_image, convert_heic_to_jpeg, encode_filename giữ nguyên như script bạn cung cấp)

def convert_heic_to_jpeg(heic_path, jpeg_path):
    """Chuyển đổi nội dung file .heic và lưu vào đường dẫn đích."""
    try:
        heif_file = pyheif.read(heic_path)
        image = Image.frombytes(
            heif_file.mode, heif_file.size, heif_file.data,
            "raw", heif_file.mode, heif_file.stride,
        )
        image.save(jpeg_path, format="JPEG", quality=95)
        return True
    except Exception as e:
        print(f"Lỗi chuyển đổi .heic: {os.path.basename(heic_path)} — {e}")
        return False

def is_valid_image(path):
    """Kiểm tra xem một file có phải là ảnh hợp lệ và không bị hỏng."""
    try:
        with Image.open(path) as img:
            img.verify()
        return True
    except Exception:
        return False

def encode_filename(original_name, split, digit, counter, ext):
    """Mã hóa tên tệp để đảm bảo duy nhất và có cấu trúc."""
    base_name = os.path.splitext(original_name)[0].lower().replace('x', '').replace('d', '')
    unique_id = f"{counter:04d}"
    return f"{split}_{digit}_{base_name}_{unique_id}{ext}"

In [ ]:
#================================================================
# --- B. KỊCH BẢN XỬ LÝ CHÍNH ---
#================================================================

# --- 0. CẤU HÌNH ---
# Nguồn dữ liệu từ các thư mục sinh viên
# data_dir = '/kaggle/working/repos' 
# Đích đến là thư mục data mới, có cấu trúc train/val
output_path = '/kaggle/working/digits_data_final'
train_ratio = 0.8
random.seed(42) 

for split in ['train', 'val']:
    for i in range(10): 
        os.makedirs(os.path.join(output_path, split, str(i)), exist_ok=True)



# --- 1. THU THẬP VÀ PHÂN LOẠI FILE BAN ĐẦU ---
student_folders = sorted(list(set(glob.glob(f'{data_dir}/*/hand_written_digit/??52????'))))
digit_files = defaultdict(list)
invalid_files_by_student = {}
all_files_scanned = 0

print(f"Bắt đầu quét {len(student_folders)} thư mục sinh viên...")
print("-" * 40)

for folder_path in student_folders:
    mssv = os.path.basename(folder_path)
    invalid_files_with_reason = []
    
    for item_name in os.listdir(folder_path):
        source_path = os.path.join(folder_path, item_name)
        if os.path.isfile(source_path):
            all_files_scanned += 1
            
            # Kiểm tra tên file
            match = re.match(r'^(\d+)_.*', item_name)
            if not match:
                # Cập nhật lại lý do lỗi cho chính xác hơn
                invalid_files_with_reason.append(f"{item_name} (Lỗi: Tên file không bắt đầu bằng chữ số)")
                continue
            
            label = match.group(1)
            # Kiểm tra nội dung file
            is_heic = item_name.lower().endswith(('.heic', '.heif'))
            if not is_heic and not is_valid_image(source_path):
                invalid_files_with_reason.append(f"{item_name} (Lỗi: File ảnh bị hỏng hoặc định dạng không hỗ trợ)")
                continue

            digit_files[label].append(source_path)

    if invalid_files_with_reason:
        invalid_files_by_student[mssv] = invalid_files_with_reason

# --- 2. PHÂN CHIA TRAIN/VAL VÀ XỬ LÝ ---
print("Bắt đầu phân chia train/val và xử lý file...")
used_filenames = set()
files_processed_count = 0
files_deleted_after_copy = 0

for digit, files in digit_files.items():
    random.shuffle(files)
    split_idx = int(len(files) * train_ratio)
    train_files = files[:split_idx]
    val_files = files[split_idx:]

    counter = 1 # Reset counter cho mỗi label mới

    for split, split_files in [('train', train_files), ('val', val_files)]:
        for file_path in split_files:
            try:
                original_filename = os.path.basename(file_path)
                _ , extension = os.path.splitext(original_filename)

                # Tạo tên file mới
                new_name = encode_filename(original_filename, split, digit, counter, extension)
                
                # Logic an toàn để tránh trùng lặp (dù counter đã xử lý)
                if new_name in used_filenames: 
                    new_name = encode_filename(f"{original_filename}-dup", split, digit, counter, extension)
                used_filenames.add(new_name)
                
                counter += 1
                
                destination_path = os.path.join(output_path, split, digit, new_name)
                
                # Xử lý HEIC hoặc sao chép
                if original_filename.lower().endswith(('.heic', '.heif')):
                    # LƯU Ý: Chuyển đổi nội dung thành JPEG nhưng vẫn lưu với đuôi file gốc theo yêu cầu
                    convert_heic_to_jpeg(file_path, destination_path)
                else:
                    shutil.copy2(file_path, destination_path)
                
                # Kiểm tra lại file ở đích
                if not is_valid_image(destination_path):
                    os.remove(destination_path)
                    files_deleted_after_copy += 1
                else:
                    files_processed_count += 1
            
            except Exception as e:
                print(f"🛑 Lỗi xử lý file không xác định: {file_path} — {e}")

# --- 3. BÁO CÁO VÀ ĐÓNG GÓI ---
print("\n" + "=" * 40)
print("TỔNG KẾT TOÀN BỘ")
print("=" * 40)
total_invalid_files = sum(len(v) for v in invalid_files_by_student.values())
print(f"Tổng số file đã quét: {all_files_scanned}")
print(f"  - Số file hợp lệ đã xử lý thành công: {files_processed_count}")
print(f"  - Số file không hợp lệ (bị loại từ đầu): {total_invalid_files}")
print(f"  - Số file bị xóa sau khi xử lý (do lỗi): {files_deleted_after_copy}")

if invalid_files_by_student:
    print("\n--- DANH SÁCH CÁC FILE KHÔNG HỢP LỆ BAN ĐẦU ---")
    for mssv, files_with_reasons in invalid_files_by_student.items():
        print(f"  MSSV: {mssv}")
        for entry in files_with_reasons:
            print(f"    - {entry}")


Bắt đầu quét 66 thư mục sinh viên...
----------------------------------------
Bắt đầu phân chia train/val và xử lý file...
🛑 Lỗi xử lý file không xác định: /kaggle/working/repos/Lochke_CS114_Handwritten_Digit_Classification.git/hand_written_digit/22520717/10_3.jpg — [Errno 2] No such file or directory: '/kaggle/working/digits_data_final/train/10/train_10_10_3_0001.jpg'
🛑 Lỗi xử lý file không xác định: /kaggle/working/repos/Lochke_CS114_Handwritten_Digit_Classification.git/hand_written_digit/22520717/10_2.jpg — [Errno 2] No such file or directory: '/kaggle/working/digits_data_final/train/10/train_10_10_2_0002.jpg'
🛑 Lỗi xử lý file không xác định: /kaggle/working/repos/Lochke_CS114_Handwritten_Digit_Classification.git/hand_written_digit/22520717/10_4.jpg — [Errno 2] No such file or directory: '/kaggle/working/digits_data_final/train/10/train_10_10_4_0003.jpg'
🛑 Lỗi xử lý file không xác định: /kaggle/working/repos/Lochke_CS114_Handwritten_Digit_Classification.git/hand_written_digit/225207